In [1]:
# === Setup for Anti-Cheating Proctoring System (Late Fusion) ===
# Models: MediaPipe (Face + Gaze), YOLOv8 (Object Detection), ArcFace (Face ID)

import cv2
import mediapipe as mp
import torch
from ultralytics import YOLO
import numpy as np

# ------------------------
# Load Models
# ------------------------

# 1. MediaPipe Face Mesh + Iris for gaze
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# 2. YOLOv8 for object detection (detect phones)
yolo_model = YOLO("yolov8n.pt")  # Or your custom-trained cheating detector

# 3. ArcFace/FaceNet for face identity verification (stubbed here)
def verify_identity(face_crop):
    # Placeholder: should compare embeddings to enrolled ID
    return True

# ------------------------
# Process Frame
# ------------------------
def analyze_frame(frame):
    results = {}

    # ---------- Face Detection ----------
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_result = face_detection.process(rgb)

    results['face_present'] = bool(face_result.detections)

    # ---------- Gaze Estimation ----------
    gaze_off_screen = False
    if face_result.detections:
        mesh_result = face_mesh.process(rgb)
        if mesh_result.multi_face_landmarks:
            for landmarks in mesh_result.multi_face_landmarks:
                left_eye = np.array([
                    landmarks.landmark[474].x,
                    landmarks.landmark[474].y
                ])
                if left_eye[0] < 0.3 or left_eye[0] > 0.7:
                    gaze_off_screen = True

    results['gaze_off_screen'] = gaze_off_screen

    # ---------- Object Detection (Phone) ----------
    detections = yolo_model(frame, verbose=False)[0]
    phone_detected = False
    for det in detections.boxes:
        cls = int(det.cls[0])
        if yolo_model.names[cls].lower() in ["cell phone", "phone"]:
            phone_detected = True
            break
    results['phone_detected'] = phone_detected
    
    

    # ---------- Identity Check ----------
    results['identity_verified'] = verify_identity(frame)  # Stub

    return results

# ------------------------
# Run Live Webcam Check
# ------------------------
def run_proctoring():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error opening webcam.")
        return

    print("Proctoring started. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = analyze_frame(frame)

        status = "✅ Normal"
        color = (0, 255, 0)
        if not results['face_present'] or not results['identity_verified'] or results['phone_detected'] or results['gaze_off_screen']:
            status = "🚨 Cheating Detected"
            color = (0, 0, 255)

        cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow("Proctoring", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# ------------------------
# Run System
# ------------------------
if __name__ == "__main__":
    run_proctoring()


Proctoring started. Press 'q' to quit.


In [4]:
# === Setup for Anti-Cheating Proctoring System (Late Fusion with Weights) ===
# Models: MediaPipe (Face + Gaze), YOLOv8 (Object Detection), ArcFace (Face ID)

import cv2
import mediapipe as mp
import torch
from ultralytics import YOLO
import numpy as np

# ------------------------
# Load Models
# ------------------------

# 1. MediaPipe Face Mesh + Iris for gaze
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# 2. YOLOv8 for object detection (detect phones)
yolo_model = YOLO("yolov8n.pt")  # Or your custom-trained cheating detector

# 3. ArcFace/FaceNet for face identity verification (stubbed here)
def verify_identity(face_crop):
    # Placeholder: should compare embeddings to enrolled ID
    return True

# ------------------------
# Process Frame
# ------------------------
def analyze_frame(frame):
    results = {}

    # ---------- Face Detection ----------
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_result = face_detection.process(rgb)

    results['face_present'] = bool(face_result.detections)

    # ---------- Gaze Estimation ----------
    gaze_off_screen = False
    if face_result.detections:
        mesh_result = face_mesh.process(rgb)
        if mesh_result.multi_face_landmarks:
            for landmarks in mesh_result.multi_face_landmarks:
                left_eye = np.array([
                    landmarks.landmark[474].x,
                    landmarks.landmark[474].y
                ])
                if left_eye[0] < 0.3 or left_eye[0] > 0.7:
                    gaze_off_screen = True

    results['gaze_off_screen'] = gaze_off_screen

    # ---------- Object Detection (Phone) ----------
    detections = yolo_model(frame, verbose=False)[0]
    phone_detected = False
    for det in detections.boxes:
        cls = int(det.cls[0])
        if yolo_model.names[cls].lower() in ["cell phone", "phone"]:
            phone_detected = True
            break
    results['phone_detected'] = phone_detected

    # ---------- Identity Check ----------
    results['identity_verified'] = verify_identity(frame)  # Stub

    return results

# ------------------------
# Run Live Webcam Check
# ------------------------
def run_proctoring():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error opening webcam.")
        return

    print("Proctoring started. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = analyze_frame(frame)

        # ------------------------
        # Weighted Score Calculation
        # ------------------------

        # Assign weights
        w_face_absence = 0.25
        w_identity_mismatch = 0.25
        w_phone_detected = 0.3
        w_gaze_off_screen = 0.2

        # Boolean -> 0 or 1
        face_score = int(not results['face_present'])
        identity_score = int(not results['identity_verified'])
        phone_score = int(results['phone_detected'])
        gaze_score = int(results['gaze_off_screen'])

        # Weighted cheating score
        cheating_score = (
            w_face_absence * face_score +
            w_identity_mismatch * identity_score +
            w_phone_detected * phone_score +
            w_gaze_off_screen * gaze_score
        )

        CHEATING_THRESHOLD = 0.3  # Tune as needed

        if cheating_score > CHEATING_THRESHOLD:
            status = f"🚨 Cheating ({cheating_score:.2f})"
            color = (0, 0, 255)
        else:
            status = f"✅ Normal ({cheating_score:.2f})"
            color = (0, 255, 0)

        cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        cv2.imshow("Proctoring", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# ------------------------
# Run System
# ------------------------
if __name__ == "__main__":
    run_proctoring()

Proctoring started. Press 'q' to quit.
